In [1]:
%cd ..

/workspaces/MariaDB4p


## Initiate and start mariadb server

In [2]:
from MariaDB4p.mariadb_wrapper import MariaDBWrapper

In [9]:
wrapper = MariaDBWrapper(port=3307)

2024-10-28 05:33:29.503 | DEBUG    | MariaDB4p.check_jdk:is_jdk_installed:12 - current_dir:/workspaces/MariaDB4p/MariaDB4p
2024-10-28 05:33:29.513 | INFO     | MariaDB4p.check_jdk:is_jdk_inpath:39 - JDK has not been found in system path.
2024-10-28 05:33:29.513 | INFO     | MariaDB4p.check_jdk:is_jdk_installed:16 - jdk has been installed to:/home/vscode/.jdks/jdk-17.0.13+11
2024-10-28 05:33:29.514 | INFO     | MariaDB4p.check_jdk:install_jdk_if_missing:61 - Skipping JDK installation.
2024-10-28 05:33:29.515 | INFO     | MariaDB4p.download_jars:download_maria4j_jars:257 - Dependencies already downloaded. Skipping download.
2024-10-28 05:33:29.513 | INFO     | MariaDB4p.check_jdk:is_jdk_inpath:39 - JDK has not been found in system path.
2024-10-28 05:33:29.513 | INFO     | MariaDB4p.check_jdk:is_jdk_installed:16 - jdk has been installed to:/home/vscode/.jdks/jdk-17.0.13+11
2024-10-28 05:33:29.514 | INFO     | MariaDB4p.check_jdk:install_jdk_if_missing:61 - Skipping JDK installation.
2024

In [10]:
wrapper.start_server()

2024-10-28 05:33:33.919 | INFO     | MariaDB4p.mariadb_wrapper:start_server:80 - MariaDB server started on port 3307.


### Make a test database "db"

In [11]:
wrapper.db.createDB('db')

### Use pymysql to run some queries

In [12]:
import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             database='db',
                             port=3307
                             )

In [13]:
create_sql='''CREATE TABLE `users` (
    `id` int(11) NOT NULL AUTO_INCREMENT,
    `email` varchar(255) COLLATE utf8_bin NOT NULL,
    `password` varchar(255) COLLATE utf8_bin NOT NULL,
    PRIMARY KEY (`id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin
AUTO_INCREMENT=1 ;'''

In [14]:
with connection.cursor() as cursor:
    cursor.execute(create_sql)
    connection.commit()


In [15]:
with connection.cursor() as cursor:
    # Create a new record
    sql = "INSERT INTO `users` (`email`, `password`) VALUES (%s, %s)"
    cursor.execute(sql, ('webmaster@python.org', 'very-secret'))

# connection is not autocommit by default. So you must commit to save
# your changes.
connection.commit()

with connection.cursor() as cursor:
    # Read a single record
    sql = "SELECT `id`, `password` FROM `users` WHERE `email`=%s"
    cursor.execute(sql, ('webmaster@python.org',))
    result = cursor.fetchone()
    print(result)

(1, 'very-secret')


### Shutdown server

In [16]:
wrapper.stop_server()

2024-10-28 05:33:54.270 | INFO     | MariaDB4p.mariadb_wrapper:stop_server:92 - MariaDB server stopped.


### Clean up test database files

In [17]:
import shutil
shutil.rmtree(wrapper.base_dir)